## Is the memory bandwidth the bottleneck in RLE using x86 architectures?

The following benchmarks were done on a c7i.xlarge machine on AWS with the following cpu infos: 

```
Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  4
  On-line CPU(s) list:   0-3
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) Platinum 8488C
    CPU family:          6
    Model:               143
    Thread(s) per core:  2
    Core(s) per socket:  2
    Socket(s):           1
    Stepping:            8
    BogoMIPS:            4800.00
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc arch_perfmon rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq p
                         ni pclmulqdq monitor ssse3 fma cx16 pdcm pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp ibrs_enhanced fsgsbase tsc_adj
                         ust bmi1 avx2 smep bmi2 erms invpcid avx512f avx512dq rdseed adx smap avx512ifma clflushopt clwb avx512cd sha_ni avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves avx_vnni avx512_bf16 wbnoinvd ida arat avx512vbmi umip pku ospke w
                         aitpkg avx512_vbmi2 gfni vaes vpclmulqdq avx512_vnni avx512_bitalg tme avx512_vpopcntdq rdpid cldemote movdiri movdir64b md_clear serialize amx_bf16 avx512_fp16 amx_tile amx_int8 flush_l1d arch_capabilities

...

Caches (sum of all):
  L1d:                   96 KiB (2 instances)
  L1i:                   64 KiB (2 instances)
  L2:                    4 MiB (2 instances)
  L3:                    105 MiB (1 instance)

  ...

```

First we want to calculate an lower bound for doing our analysis. For calculating our lower bound (dominated by IO) we calculate the time that is needed to read the two values and runlength arrays and to write the resulting array back to memory. Therefore i modified the membw.cpp script that just does that with a single core (see membw_rle_lower.cpp). The two parts that calculate this are the following: 

- calculate the time to write n uint32_t (-> sizeof(uint32_t) * n bytes to memory)
- calculate the time to read the two runlength/value lists from memory (this is based on the compression factor; in our setup we tested with data containing only 64, 128, 256, 512 fixed sized runlengths)